# 🧠 Differential Privacy (DP-SGD) vs Normal SGD
### For Pretrained DeepNN Model — BSAVNN Privacy Comparison

This notebook compares **Standard SGD** and **DP-SGD** using the Opacus library, with visualizations for:
- Accuracy vs Privacy (ε)
- Training Loss Curves

**Author:** Sushanth

---

In [ ]:
# ================================================================
# Imports
# ================================================================
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.optim import Adam
from opacus import PrivacyEngine
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import matplotlib.pyplot as plt
import os

# Ensure figures directory exists
os.makedirs('figures', exist_ok=True)
print('✅ Libraries imported and figures directory ready.')

In [ ]:
# ================================================================
# DeepNN Model Definition
# ================================================================
class DeepNN(nn.Module):
    def __init__(self, input_dim):
        super(DeepNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 48),
            nn.ReLU(),
            nn.Linear(48, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)


def load_model(input_dim, path="lung_cancer_model.pth"):
    model = DeepNN(input_dim)
    try:
        checkpoint = torch.load(path, map_location="cpu")
        model.load_state_dict(checkpoint)
        print(f"✅ Pretrained model loaded from {path}")
    except Exception as e:
        print(f"⚠️ Could not load pretrained weights: {e}. Using random weights.")
    return model

In [ ]:
# ================================================================
# Dataset Preparation
# ================================================================
def prepare_data():
    # Replace this with your dataset
    X = np.random.rand(500, 20)
    y = np.random.randint(0, 2, 500)

    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

    dataset = TensorDataset(X_tensor, y_tensor)

    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_set, val_set = random_split(dataset, [train_size, val_size])

    return train_set, val_set, X.shape[1]

train_set, val_set, input_dim = prepare_data()
print(f"✅ Data prepared with input_dim = {input_dim}")

In [ ]:
# ================================================================
# Train Function (Supports Normal & DP-SGD)
# ================================================================
def train_model(model, train_loader, optimizer, epochs=5, dp=False):
    model.train()
    epsilons, losses = [], []

    for epoch in range(epochs):
        total_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = F.binary_cross_entropy(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        losses.append(total_loss / len(train_loader))

        if dp:
            epsilon, best_alpha = optimizer.privacy_engine.get_privacy_spent(delta=1e-5)
            epsilons.append(epsilon)
            print(f"Epoch {epoch+1}/{epochs} | DP Loss: {losses[-1]:.4f} | ε={epsilon:.2f} | α={best_alpha}")
        else:
            print(f"Epoch {epoch+1}/{epochs} | Normal Loss: {losses[-1]:.4f}")

    return model, losses, epsilons

In [ ]:
# ================================================================
# Evaluation Function
# ================================================================
def evaluate_model(model, val_loader, label=""):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            preds = (model(X_batch) > 0.5).float()
            all_preds.extend(preds.flatten().tolist())
            all_labels.extend(y_batch.flatten().tolist())
    acc = accuracy_score(all_labels, all_preds)
    print(f"\n🔹 {label} Accuracy: {acc:.4f}")
    print(classification_report(all_labels, all_preds))
    return acc

In [ ]:
# ================================================================
# Main Comparison: Normal SGD vs DP-SGD
# ================================================================
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)

# -------- Normal Training --------
normal_model = load_model(input_dim)
normal_optimizer = Adam(normal_model.parameters(), lr=1e-3)
normal_model, normal_losses, _ = train_model(normal_model, train_loader, normal_optimizer, epochs=8)
normal_acc = evaluate_model(normal_model, val_loader, label="Normal SGD")

# -------- DP-SGD Training --------
dp_model = load_model(input_dim)
dp_optimizer = Adam(dp_model.parameters(), lr=1e-3)

privacy_engine = PrivacyEngine(
    dp_model,
    batch_size=32,
    sample_size=len(train_set),
    alphas=[10, 100],
    noise_multiplier=1.1,
    max_grad_norm=1.0,
)
privacy_engine.attach(dp_optimizer)

dp_model, dp_losses, epsilons = train_model(dp_model, train_loader, dp_optimizer, epochs=8, dp=True)
dp_acc = evaluate_model(dp_model, val_loader, label="DP-SGD")

In [ ]:
# ================================================================
# Visualization: Accuracy vs Privacy and Loss Comparison
# ================================================================
plt.figure(figsize=(7, 5))
plt.plot(epsilons, [dp_acc] * len(epsilons), 'o-', label="DP-SGD (ε vs Accuracy)")
plt.axhline(y=normal_acc, color='r', linestyle='--', label=f"Normal SGD Accuracy = {normal_acc:.2f}")
plt.xlabel("Privacy Budget (ε)")
plt.ylabel("Validation Accuracy")
plt.title("Accuracy vs Privacy (DP-SGD vs Normal SGD)")
plt.legend()
plt.grid(True)
plt.savefig("figures/privacy_accuracy_tradeoff.png")
plt.show()

plt.figure(figsize=(7, 5))
plt.plot(normal_losses, label="Normal SGD")
plt.plot(dp_losses, label="DP-SGD")
plt.xlabel("Epochs")
plt.ylabel("Training Loss")
plt.title("Training Loss Comparison (DP-SGD vs Normal SGD)")
plt.legend()
plt.grid(True)
plt.savefig("figures/loss_comparison.png")
plt.show()

print("✅ Results and plots saved in 'figures/' directory.")